<img src="logo2.png"></img>

# Finalチャレンジ 解説と講評

***

# 最終問題：自治体のコンビニ出店プランを提示せよ
東京のZ市は11の区域からなる自治体で、すでに4社のコンビニ(A社,B社,C社,D社)が本社の1店舗を別々の区域に展開しています。  
現在の出店状況は地図の通りです。地図上の各ノードはそれぞれの区域を表しており、地図上のエッジは、エッジの両端の区域が隣り合っていることを表しています。

<img src="./tokyo_map_pic.png" width="700">

市長のあなたは、残りの7区域にもコンビニを誘致しようとしましたが、４社から以下の条件が提示されました。
* 1つの区域に出店出来るのは1社のコンビニのみ
* 自社のコンビニは、隣接する区域に自社のコンビニが既に出店している場合は出店しない。

あなたはこれらの条件を満たす出店案を提示できるでしょうか？  
**Week2,3で使ったグローバーのアルゴリズムをiteration回数＝5で用いて、条件を満たす全ての出店案を列挙してみてください。**
<br/><br/>
***

# 解説
(こちらでご紹介する解説はひとつの解法のサンプルであり、唯一の解法ではないことを予めご承知おきください。）

Finalチャレンジは今までと比べて問題の規模が大きいですが、アルゴリズムの流れはweek2と変わりません。

|step|内容|
|:---:|:---:|
|Step1|入力の重ね合わせ状態を作る|
|Step2|問題の制約に基づくオラクルを構成する|
|Step3|diffusion|
|Step4|測定|

## 量子レジスタ
量子レジスタ内のデータ割り当ては以下のようになります。

|量子ビット|量子ビット数|機能|
|:---:|:---:|:---:|
|q[0]-q[13]|14|各地域(7)に出店しているコンビニの情報(1地域あたり$\log_2{4}$)|
|q[14]-q[26]|13|オラクル内の記憶領域|
|q[27]|1|オラクルの最終結果を格納する量子ビット|
|q[28]-q[31]|4|オラクルなどのためのアンシラ|


## Step1: 入力の重ね合わせ状態を作る
まず各種モジュールをインポートして、量子回路を用意します。

In [14]:
import numpy as np
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit import IBMQ, execute

qr = QuantumRegister(32)
cr = ClassicalRegister(14)
qc = QuantumCircuit(qr,cr)

各コンビニ(q[0]からq[13])の状態を4社の重ね合わせ状態にします。

このとき、A,B,C,Dの各本社と隣接している地域に関しては重ね合わせる会社を減らしておきます。 これは、解の取りうる空間を小さくすることに役立ちます。

例えば、地域3はA社の本社と隣接しているので、
$|00>$を取り除いて$|01>$と$|10>$と$|11>$の重ね合わせ状態にします。

また、地域2はA社、C社の本社と隣接しているので、
$|00>$と$|10>$を取り除いて$|01>$と$|11>$の重ね合わせ状態にします。

In [2]:
theta = 2 * np.arccos(1 / np.sqrt(3))

In [3]:
def initialize_districts(qc, qr):
    #district0 A 
    qc.ry(theta,qr[0])
    qc.ch(qr[0],qr[1])
    qc.x(qr[1])

    #district1 B
    qc.ry(theta,qr[2])
    qc.ch(qr[2],qr[3])

    #district2 A,C
    qc.h(qr[4])
    qc.x(qr[5])

    #district3 A
    qc.ry(theta,qr[6])
    qc.ch(qr[6],qr[7])
    qc.x(qr[7])

    #district4 B
    qc.ry(theta,qr[8])
    qc.ch(qr[8],qr[9])

    #district5 D
    qc.ry(theta,qr[10])
    qc.ch(qr[10],qr[11])
    qc.x(qr[10])

    #district6 D
    qc.ry(theta,qr[12])
    qc.ch(qr[12],qr[13])
    qc.x(qr[12])

## Step2: 問題の制約に基づくオラクルを構成する

step.1で各本社の隣接状態に関しては状態を入力したので、残っている条件は以下のような13本のエッジになります。

<img src="./fig/white_map.png" width="400"></img>

In [4]:
edge_list = [[0,1],[0,2],[0,3],[1,3],[1,4],[2,3],[2,5],[2,6],[3,4],[3,5],[3,6],[4,6],[5,6]]

隣接している(辺がある)頂点のペアごとに、それぞれ制約を満たしているか(違うコンビニ会社の出店になっているか)確認するオラクルを作ります。

例えば、ある地域nとその隣の地域n+1の両方の地域にD社が出店してしまっている状態を見つけるには次のような回路を組みます。

<img src="./fig/oracle.png" width="400"></img>

なお、A社(00)のような状態に0を含むコンビニの隣接条件に関しては、一度反転してからCCCCXゲートを実行し、もう一度反転して戻す操作を行います。

このような回路をAからD社全てに関して作れば、地域nとn+1の隣接条件が満たされている(異なるコンビニが出店している)かどうかの確認ができます(白い丸はビット反転させたコントロールを意味します)。


<img src="./fig/konbini_oracle.png" width="700"></img>

上記の回路は実は下の回路で置き換えることができます（詳細についてはページ下部のTipsを参照。興味がある人は真理値表を書いて確かめてみてください）。下の回路を用いて隣接した地域に違うコンビニが出店しているかチェックします。

<img src="./fig/konbini_oracle_simple.png" width="600"></img>

今回の実装ではccxやcccxの代わりに、よりコストが低いrccxやrcccxを用いています。rccxやrcccxを用いると通常はphaseが異なる部分が出てきてしまいますが、uncomputeする際にinverseを適用することでphaseが元にもどるので、問題なく利用することができます。rccxやrcccxの詳細については下記の論文を参照してください。  
[On the advantages of using relative phase Toffolis with an application to multiple control Toffoli optimization](https://arxiv.org/pdf/1508.03273.pdf)

In [5]:
def color_check(qc, data, district1,district2, result):
    data_qubits = [data[district1*2], data[(district1*2)+1], data[district2*2], data[(district2*2)+1]]
    qc.barrier()
    qc.rccx(data_qubits[0], data_qubits[1], result)
    qc.barrier()
    qc.x(data_qubits[2])
    qc.rccx(data_qubits[1], data_qubits[2], result)
    qc.x(data_qubits[2])

    qc.barrier()
    qc.x(data_qubits[2])
    qc.x(data_qubits[3])
    qc.rccx(data_qubits[2], data_qubits[3], result)
    qc.x(data_qubits[2])
    qc.x(data_qubits[3])
    qc.barrier()
    
    qc.x(data_qubits[3])    
    qc.rccx(data_qubits[0], data_qubits[3], result)
    qc.x(data_qubits[3])    
    qc.barrier()

In [6]:
def color_check_inverse(qc, data, district1,district2, result):
    data_qubits = [data[district1*2], data[(district1*2)+1], data[district2*2], data[(district2*2)+1]]
    
    qc.barrier()
    qc.x(data_qubits[3])    
    qc.rccx(data_qubits[0], data_qubits[3], result)
    qc.x(data_qubits[3])    
    qc.barrier()
    
    qc.barrier()
    qc.x(data_qubits[2])
    qc.x(data_qubits[3])
    qc.rccx(data_qubits[2], data_qubits[3], result)
    qc.x(data_qubits[2])
    qc.x(data_qubits[3])
    qc.barrier()
    
    qc.x(data_qubits[2])
    qc.rccx(data_qubits[1], data_qubits[2], result)
    qc.x(data_qubits[2])
    
    qc.barrier()
    qc.rccx(data_qubits[0], data_qubits[1], result)
    qc.barrier()

In [7]:
def rcccx_inverse(circ,control_0,control_1,control_2,target):
    circ.u2(0, np.pi, target)  
    circ.u1(np.pi / 4, target)  
    circ.cx(control_2, target)
    circ.u1(-np.pi / 4, target)
    circ.u2(0, np.pi, target)
    circ.u1(np.pi / 4, target)
    circ.cx(control_1, target)
    circ.u1(-np.pi / 4, target)
    circ.cx(control_0, target)
    circ.u1(np.pi / 4, target)
    circ.cx(control_1, target)
    circ.u1(-np.pi / 4, target)
    circ.cx(control_0, target)
    circ.u2(0, np.pi, target) 
    circ.u1(np.pi / 4, target)
    circ.cx(control_2, target)
    circ.u1(-np.pi / 4, target)
    circ.u2(0, np.pi, target) 

13本のエッジに対して`color_check()`を実行します。13個の`color_check()`の結果のANDを`mct`を用いて取得し、それをオラクルの符号反転用の量子ビットに保存します。

In [13]:
def correct_result(qc, result, anc, register):
    # correct results of color_check() and store in anc
    for i in range(4):
        qc.rcccx(result[3*i], result[3*i+1], result[3*i+2], anc[i])
        qc.barrier()
    
    # use cccccx that has result[13] and anc[0] to anc[3] as its control bits, and register as its target bit to flip the phase of correct answers.
    mct_control=[result[12]]+anc
    qc.mct(mct_control, register, result[0:3], mode='basic-dirty-ancilla')
    
    # uncompute corrected results  
    for i in reversed(range(4)):
        rcccx_inverse(qc,result[3*i], result[3*i+1], result[3*i+2], anc[i])
        qc.barrier()

`color_check()`と`collect_result()`を組み合わせて`oracle()`を作成します。`color_check()`は隣接する地域に同じコンビニが出店していた場合に1を返すので、Xゲートを用いてresultの初期状態を$|1>$にしておきます。

In [9]:
def oracle(qc, data, result, anc, register):
    result_index = 0
    
    #initialize memory_for_result to |1>
    qc.x(result)
        
    #color check for each edge
    for edge in edge_list:
        color_check(qc, data, edge[0],edge[1], result[result_index])
        result_index += 1

    qc.barrier()

    #collect results of each edge
    correct_result(qc, result, anc, register)
    qc.barrier()
    
    #inversed color check
    result_index = 0 
    for edge in edge_list:
        color_check_inverse(qc, data, edge[0],edge[1], result[result_index])
        result_index += 1
    qc.barrier()
    
    #reinitilize memory_for_result to |0>
    qc.x(result)

## Step3: diffusion

week2にある通り、グローバーの回路は、以下の要素から構成されるのでした。
1. 反転のオラクル$U_w$
2. 初期状態$|s>$に対する反転$U_s$

<img src="./fig/step2.png" width="700">

このstepでは、初期状態$|s>$に対して反転を行う回路(diffusion回路)$U_s$を作成します。

week2では、アダマールゲートを用いて、全状態の均等な重ね合わせ状態を作りましたが、今回の問題では初期状態にも制約を入れるので、diffusionに関しても少しだけ変更が必要です。

初期状態$|s>$に対する反転は、$U_{s} = 2|s> <s| - I $ のような形で表せるものであったことを思い出してください。

step1で作った、制約のある初期状態を生成する回路を$U_{const}$と置きます。
すると、
$$U_s = 2|s> <s| - I \\= 2U_{const}|0><0|U_{const}^\dagger - I \\=-U_{const}(I-2|0><0|)U^\dagger$$
であるので、$U_{const}$と逆の操作$U_{const}^\dagger$が必要であることがわかります。

$U_{const}^\dagger$を実際に作ってみると以下のようになります。

In [10]:
def initialize_districts_inverse(qc, qr):
    #district0 A inverse
    qc.x(qr[1])
    qc.ch(qr[0],qr[1])
    qc.ry(-theta,qr[0])

    #district1 B inverse
    qc.ch(qr[2],qr[3])
    qc.ry(-theta,qr[2])

    #district2 A,C inverse
    qc.x(qr[5])
    qc.h(qr[4])

    #district3 A  inverse
    qc.x(qr[7])
    qc.ch(qr[6],qr[7])
    qc.ry(-theta,qr[6])

    #district4 B  inverse
    qc.ch(qr[8],qr[9])
    qc.ry(-theta,qr[8])

    #district5 D  inverse
    qc.x(qr[10])
    qc.ch(qr[10],qr[11])
    qc.ry(-theta,qr[10])

    #district6 D  inverse
    qc.x(qr[12])
    qc.ch(qr[12],qr[13])
    qc.ry(-theta,qr[12])

In [11]:
def inversion_about_average(circuit, register):
    """Apply inversion about the average step of Grover's algorithm."""
    initialize_districts_inverse(circuit, register)
    circuit.x(register[0:14])
    
    circuit.h(register[13])
    circuit.mct(register[0:13],register[13],register[14:26])
    circuit.h(register[13])

    circuit.x(register[0:14])
    initialize_districts(circuit, register)

入力の量子ビットとphase filp用に使う量子ビットを初期化し、イテレーションを5回繰り返します。

In [15]:
# Initialize 
initialize_districts(qc, qr)

# Initialize a qubit for phase flip
qc.x(qr[27])
qc.h(qr[27])

# Grover iteration
for i in range(5):
    oracle(qc, qr[0:14], qr[14:27], qr[28:32], qr[27])
    inversion_about_average(qc,qr)

# Uncompute
qc.h(qr[27])
qc.x(qr[27])

## Step4: 測定
各地域の状態をマッピングしてある量子ビットを観測します。

In [16]:
%%time
qc.measure(qr[0:14],cr[0:14])

provider = IBMQ.load_account()
backend = provider.get_backend('ibmq_qasm_simulator')
job = execute(qc, backend=backend, shots=8000, seed_simulator=12345, backend_options={"fusion_enable":True})
result = job.result()
count = result.get_counts()

Error checking job status using websocket, retrying using HTTP.


CPU times: user 26.3 s, sys: 831 ms, total: 27.1 s
Wall time: 1h 12min 16s


測定結果は膨大なので、ソートして15番目までを取り出します。

In [17]:
score_sorted = sorted(count.items(), key=lambda x:x[1], reverse=True)
final_score = score_sorted[0:15]
final_score

[('00010111100001', 546),
 ('00011110110001', 531),
 ('10000001111110', 523),
 ('01001110110001', 523),
 ('00010110111101', 515),
 ('01000010111101', 509),
 ('00010110110001', 506),
 ('00101101110010', 501),
 ('10001101110010', 496),
 ('00000001110101', 9),
 ('01010101110001', 8),
 ('00010001101111', 8),
 ('10000001100110', 8),
 ('00100011110011', 8),
 ('10100101101111', 7)]

９番目までの測定結果が優位に増幅しているので、解とみなして確かめてみます。

In [34]:
#sort count
count_sorted = sorted(count.items(), key=lambda x:x[1], reverse=True)

# collect answers with Top 9 probability
ans_list = count_sorted[0:9]

# reverse ans_list
ans_reversed = []
for i in ans_list:
    ans_temp=[i[0][::-1],i[1]]
    ans_reversed.append(ans_temp)

# convert each 2 bits into corresponding color. Add node0(0),node3(1),node8(2) and node11(3)
ans_shaped = []
for j in ans_reversed:
    ans_temp=j[0]
    nodeA = 0
    node0 = int(ans_temp[0] + ans_temp[1], 2)
    node1 = int(ans_temp[2] + ans_temp[3], 2)
    nodeB = 1
    node2 = int(ans_temp[4] + ans_temp[5], 2)
    node3 = int(ans_temp[6] + ans_temp[7], 2)
    node4 = int(ans_temp[8] + ans_temp[9], 2)
    nodeC = 2
    node5 = int(ans_temp[10] + ans_temp[11], 2)
    node6 = int(ans_temp[12] + ans_temp[13], 2)
    nodeD = 3
    nodes_color = str(nodeA) + str(node0) + str(node1) + str(nodeB) + str(node2) + str(node3) + str(node4) + str(nodeC) + str(node5) + str(node6) + str(nodeD) 
    ans_shaped.append([nodes_color,j[1]])
sorted(ans_shaped)

[['01013232013', 496],
 ['01013232103', 501],
 ['01313202013', 523],
 ['02011322203', 546],
 ['02013122203', 506],
 ['02013132023', 523],
 ['02013132203', 531],
 ['02313102023', 509],
 ['02313122203', 515]]

答えの通りに街を塗ってみると以下のようになっています。問題なく塗り分けられているようですね!
完了です!
<img src="./fig/all_answers.png" width="600"></img>

In [29]:
import json
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Unroller

# Unroll the circuit
pass_ = Unroller(['u3', 'cx'])
pm = PassManager(pass_)
new_circuit = pm.run(qc) 

# obtain gates
gates=new_circuit.count_ops()
print(gates)

OrderedDict([('u3', 6507), ('cx', 2416), ('barrier', 770), ('measure', 14)])


In [30]:
cost=gates['u3'] + 10*gates['cx']
print(cost)

30667


この回路のコストは量子コストは$30667$です。

## TIPS
`color_check()`の4つのccccxを4つのccxで置き換える方法について、4bitのgray codeは4次元の超立方体にマッピングすることができます（下図はWikipediaの[Hamming distance](https://en.wikipedia.org/wiki/Hamming_distance)より）。
<img src="./fig/4bit_hyper_cube.png" width="600"></img>

4つのccccxはそれぞれ超立方体上の頂点0000, 0101, 1010, 1111を塗ることに対応しています。ccxは超立方体上の面を塗ることに対応しているので、適切な面の組み合わせ（偶数個の面で共有している頂点は打ち消しあい、色が塗られません）を見つけることが出来れば、4つのccccxはccxの組み合わせで置き換えることができます。試してみると下記の面の組み合わせで0000, 0101, 1010, 1111を塗ることが出来ることが分かります（その他の塗られた頂点については、偶数個の面で共有されているため実際には色が塗られていないことに注意してください）。
<img src="./fig/hyper_cube_ccx.png" width="600"></img>

それぞれの面に対応するccxは以下の通りです。
<img src="./fig/konbini_oracle_hyper_cube.png" width="500"></img>

## 講評
特に優秀な成績を収められた皆さんの実装についてコメントしておきます。

**上位3チーム**
* 1位 **Whit3z**: cost = $16613$  
同時に色判定可能な地域の分割が洗練されており、Oracle部分の圧縮を実現しています。
また、複数の単一量子ビットゲートをu3ゲートでまとめたり、margolus gateを定義したりするなど、細部まで行き届いた最適化により優勝を成し遂げました。また、Whit3zさんは誰よりも早く正解しました。その際のコストは164749でしたが、ゲート削減の努力を継続し、一週間で10分の1にまで減らしたことは特筆に値します。
* 2位 **QunaVillage**: cost = $17053$  
地域情報の格納に用いる量子ビット数を削減することで問題空間を狭め、Diffusion部分の小型化と増幅率の向上を実現しています。
また、複数の辺を同時に判定することで、効率的なOracleを組み上げています。
* 3位 **IIQ@QIC**: cost = $25490$  
頂点を共有しない2つの辺のチェックを並行して進める効率的なOracle部分が設計されています。
全体的に非常に丁寧な最適化が行われており、初期状態生成（それと、対応するDiffusion部分の冒頭・末尾の操作）にもu3削減のための工夫が施されていたことが印象的でした。

======<br/>
**オラクル特別賞**  
以下のチームは、初期値の設定部分が一部不完全でしたが、オラクルやDiffusionの部分が極めて優れていたので、特別賞を設置しました。
* オラクル特別賞 **Costs > 100k**: $\approx 15737$  
多数の地域情報を組み込んだ斬新な初期状態を用いることで、問題空間の大部分を削減しています。
これにより、Oracle部分ではグラフ内のいくつかの三角形と少しのノードを確認するだけで問題解決が可能な優れたアプローチです。
* オラクル特別賞 **Sorin**: $\approx 21667$  
本稿でのcolor_checkに相当する、隣り合う地域の比較のための関数`equalsNoCcx()`が特徴的です。CXゲート2つとXゲート２つだけを使って、地域の状態を格納するために用意した量子ビットに直接比較結果を格納します。これにより、効率的かつ量子ビット数をうまく活用したOracleを構築しています。


今回のコンテストは、最初のラーニングチャレンジを含めて700人以上の方々にご参加いただきました。最終問題でみせてくださった各チームの創意工夫には私たちジャッジも本当に感動しました。特に上位チームは、ジャッジの予想をはるかに超えて量子コストを減らしてこられ、解法も素晴らしいものでした。改めて、これらの素晴らしい実装を成し遂げたチームの皆さんに拍手を送りたいと思います。おめでとうございます！
(**上位チームのサブミッションは [こちら](https://github.com/quantum-challenge/2019/tree/master/top%20ten%20submissions)からご覧いただくことができます。**)

また、今回は競技としてこそは参加しなかったものの、量子計算の基礎や新しいテクニックを習得するために参加された方も数多くいらっしゃったとお聞きしております。このコンテストをきっかけに、これからも皆さんが量子コンピューティングに親しみつづけてくだされば本望です。

この約ひと月のチャレンジ期間は、私たちジャッジにとっても学びの多い、大変刺激的で楽しい時間でした。ありがとうございます。<br/>
私たちがこのコンテストに楽しんで取り組んでいたように、皆さんがこのコンテストを楽しんで頂けていたのでしたら幸いです。

またお会いできる日まで。

IBM Quantum Challenge ジャッジ、運営事務局一同